In [ ]:
from keras.models import Sqeuntial
from keras.layers import LSTM, Dropout, Dense, Activation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()